In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import os
import sys
import matplotlib.pyplot as plt
from scipy.io import loadmat
import h5py
from scipy import signal
from math import pi
from sklearn.preprocessing import MinMaxScaler,StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold,cross_val_score,train_test_split,cross_validate,GridSearchCV
from sklearn import tree,linear_model,svm
from sklearn.preprocessing import MinMaxScaler,StandardScaler, OneHotEncoder, RobustScaler
from sklearn.metrics import r2_score
import scipy.fftpack as syfp
from sklearn.compose import ColumnTransformer

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
data_dir='./gdrive/My Drive/BMME890_MachineLearning/ProjectData'


In [0]:
data_neu=pd.read_csv(data_dir+'/allneu.csv')
data_pro=pd.read_csv(data_dir+'/allpro.csv')
data_sup=pd.read_csv(data_dir+'/allsup.csv')


In [0]:
#start with models of both fingers
y_neu=data_neu['y_Neu']
y_pro=data_pro['y_Pro']
y_sup=data_sup['y_Sup']

x_neu=data_neu[['zc','maxamp','rms','Finger']]
x_pro=data_pro[['zc','maxamp','rms','Finger']]
x_sup=data_sup[['zc','maxamp','rms','Finger']]

In [0]:
tfrm_pipeline=Pipeline([('robust_scaler', RobustScaler())])
neu_prepared=tfrm_pipeline.fit_transform(x_neu)
pro_prepared=tfrm_pipeline.fit_transform(x_pro)
sup_prepared=tfrm_pipeline.fit_transform(x_sup)


In [0]:
#neutral v neutral


rfmodel=RandomForestRegressor()
cv=KFold(n_splits=7)
rf_params={'n_estimators':[300,400],'min_samples_split':[2,5],'max_features':['sqrt'],'random_state':[42]}
rfgsneu=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfgsneu.fit(neu_prepared,y_neu)
rfgsppro=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfgsppro.fit(pro_prepared,y_pro)
rfgsssup=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfgsssup.fit(sup_prepared,y_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs

In [0]:
print(rfgsneu.best_estimator_)
print(rfgsppro.best_estimator_)
print(rfgsssup.best_estimator_)
print(rfgsneu.best_score_)
print(rfgsppro.best_score_)
print(rfgsssup.best_score_)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
     

In [0]:
#svm

svmodel=SVR()
sv_params={'tol':[0.001,.0001,.00001], 'C':[.01,.1,1]}
svgsneu=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsneu.fit(neu_prepared,y_neu)
svgsppro=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsppro.fit(pro_prepared,y_pro)
svgsssup=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsssup.fit(sup_prepared,y_sup)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1], 'tol': [0.001, 0.0001, 1e-05]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(svgsneu.best_estimator_)
print(svgsppro.best_estimator_)
print(svgsssup.best_estimator_)
print(svgsneu.best_score_)
print(svgsppro.best_score_)
print(svgsssup.best_score_)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.0001, verbose=False)
SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.0001, verbose=False)
0.650682137408441
0.6694872588195196
0.6358079468132133


In [0]:
#linear regressor


lmodel=LinearRegression()
l_params={'fit_intercept':['False','True']}
lgsneu=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsneu.fit(neu_prepared,y_neu)
lgsppro=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsppro.fit(pro_prepared,y_pro)
lgsssup=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsssup.fit(sup_prepared,y_sup)


GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid={'fit_intercept': ['False', 'True']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(lgsneu.best_estimator_)
print(lgsppro.best_estimator_)
print(lgsssup.best_estimator_)
print(lgsneu.best_score_)
print(lgsppro.best_score_)
print(lgsssup.best_score_)

LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
0.6044475993744841
0.573174579717255
0.35161353889249786


In [0]:
# index fingers
ind_neu=data_neu[data_neu['Finger']==0][['zc','maxamp','rms']]
ind_pro=data_pro[data_pro['Finger']==0][['zc','maxamp','rms']]
ind_sup=data_sup[data_sup['Finger']==0][['zc','maxamp','rms']]

indy_neu=data_neu[data_neu['Finger']==0]['y_Neu']
indy_pro=data_pro[data_pro['Finger']==0]['y_Pro']
indy_sup=data_sup[data_sup['Finger']==0]['y_Sup']

indneu_prepared=tfrm_pipeline.fit_transform(ind_neu)
indpro_prepared=tfrm_pipeline.fit_transform(ind_pro)
indsup_prepared=tfrm_pipeline.fit_transform(ind_sup)




In [0]:
rfmodel=RandomForestRegressor()
cv=KFold(n_splits=7)
rf_params={'n_estimators':[100,300,400],'min_samples_split':[2,5],'max_features':['sqrt'],'random_state':[42]}
rfindneu=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfindneu.fit(indneu_prepared,indy_neu)
rfindpro=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfindpro.fit(indpro_prepared,indy_pro)
rfindsup=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfindsup.fit(indsup_prepared,indy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs

In [0]:
print(rfindneu.best_estimator_)
print(rfindpro.best_estimator_)
print(rfindsup.best_estimator_)
print(rfindneu.best_score_)
print(rfindpro.best_score_)
print(rfindsup.best_score_)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
     

In [0]:
from sklearn.linear_model import LinearRegression
lmodel=LinearRegression()
l_params={'fit_intercept':['False','True']}
lgsindneu=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsindneu.fit(indneu_prepared,indy_neu)
lgsindppro=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsindppro.fit(indpro_prepared,indy_pro)
lgsindssup=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsindssup.fit(indsup_prepared,indy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid={'fit_intercept': ['False', 'True']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(lgsindneu.best_estimator_)
print(lgsindppro.best_estimator_)
print(lgsindssup.best_estimator_)
print(lgsindneu.best_score_)
print(lgsindppro.best_score_)
print(lgsindssup.best_score_)

LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
0.6410132207762251
0.5353992997967246
0.7603501865494349


In [0]:
svmodel=SVR()
sv_params={'tol':[0.001,.0001], 'C':[.01,.1,1]}
svgsindneu=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsindneu.fit(indneu_prepared,indy_neu)
svgsindppro=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsindppro.fit(indpro_prepared,indy_pro)
svgsindssup=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsindssup.fit(indsup_prepared,indy_sup)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1], 'tol': [0.001, 0.0001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(svgsindneu.best_estimator_)
print(svgsindppro.best_estimator_)
print(svgsindssup.best_estimator_)
print(svgsindneu.best_score_)
print(svgsindppro.best_score_)
print(svgsindssup.best_score_)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.0001, verbose=False)
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
0.6467169577338935
0.6888431695857662
0.8255291380765963


In [0]:
# middle fingers
mid_neu=data_neu[data_neu['Finger']==1][['zc','maxamp','rms']]
mid_pro=data_pro[data_pro['Finger']==1][['zc','maxamp','rms']]
mid_sup=data_sup[data_sup['Finger']==1][['zc','maxamp','rms']]

midy_neu=data_neu[data_neu['Finger']==1]['y_Neu']
midy_pro=data_pro[data_pro['Finger']==1]['y_Pro']
midy_sup=data_sup[data_sup['Finger']==1]['y_Sup']

midneu_prepared=tfrm_pipeline.fit_transform(mid_neu)
midpro_prepared=tfrm_pipeline.fit_transform(mid_pro)
midsup_prepared=tfrm_pipeline.fit_transform(mid_sup)

In [0]:
rfmodel=RandomForestRegressor()
cv=KFold(n_splits=7)
rf_params={'n_estimators':[100,300,400],'min_samples_split':[2,5],'max_features':['sqrt'],'random_state':[42]}
rfmidneu=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfmidneu.fit(midneu_prepared,midy_neu)
rfmidpro=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfmidpro.fit(midpro_prepared,midy_pro)
rfmidsup=GridSearchCV(rfmodel, rf_params,cv = cv,scoring='r2')
rfmidsup.fit(midsup_prepared,midy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs

In [0]:
print(rfmidneu.best_estimator_)
print(rfmidpro.best_estimator_)
print(rfmidsup.best_estimator_)
print(rfmidneu.best_score_)
print(rfmidpro.best_score_)
print(rfmidsup.best_score_)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
     

In [0]:
lmodel=LinearRegression()
l_params={'fit_intercept':['False','True']}
lgsmidneu=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsmidneu.fit(midneu_prepared,midy_neu)
lgsmidppro=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsmidppro.fit(midpro_prepared,midy_pro)
lgsmidssup=GridSearchCV(lmodel, l_params,cv = cv,scoring='r2')
lgsmidssup.fit(midsup_prepared,midy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid={'fit_intercept': ['False', 'True']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(lgsmidneu.best_estimator_)
print(lgsmidppro.best_estimator_)
print(lgsmidssup.best_estimator_)
print(lgsmidneu.best_score_)
print(lgsmidppro.best_score_)
print(lgsmidssup.best_score_)

LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)
0.5867802786245709
0.5397494491440197
-0.17063293136797997


In [0]:
svmodel=SVR()
sv_params={'tol':[0.001,.0001], 'C':[.01,.1,1]}
svgsmidneu=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsmidneu.fit(midneu_prepared,midy_neu)
svgsmidppro=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsmidppro.fit(midpro_prepared,midy_pro)
svgsmidssup=GridSearchCV(svmodel, sv_params,cv = cv,scoring='r2')
svgsmidssup.fit(midsup_prepared,midy_sup)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1], 'tol': [0.001, 0.0001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(svgsmidneu.best_estimator_)
print(svgsmidppro.best_estimator_)
print(svgsmidssup.best_estimator_)
print(svgsmidneu.best_score_)
print(svgsmidppro.best_score_)
print(svgsmidssup.best_score_)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
0.6036656198699644
0.6466342409362333
0.3741053870717785


In [0]:
#cross validate all models with best parameters using rmse



rfgsneuscores=cross_val_score(rfgsneu.best_estimator_,neu_prepared,y_neu,cv=cv,scoring='neg_mean_squared_error')
rfgspproscores=cross_val_score(rfgsppro.best_estimator_,pro_prepared,y_pro,cv=cv,scoring='neg_mean_squared_error')
rfgssupscores=cross_val_score(rfgsssup.best_estimator_,sup_prepared,y_sup,cv=cv,scoring='neg_mean_squared_error')
svgsneuscores=cross_val_score(svgsneu.best_estimator_,neu_prepared,y_neu,cv=cv,scoring='neg_mean_squared_error')
svgsproscores=cross_val_score(svgsppro.best_estimator_,pro_prepared,y_pro,cv=cv,scoring='neg_mean_squared_error')
svgssupscores=cross_val_score(svgsssup.best_estimator_,sup_prepared,y_sup,cv=cv,scoring='neg_mean_squared_error')
lgsneuscores=cross_val_score(lgsneu.best_estimator_,neu_prepared,y_neu,cv=cv,scoring='neg_mean_squared_error')
lgspproscores=cross_val_score(lgsppro.best_estimator_,pro_prepared,y_pro,cv=cv,scoring='neg_mean_squared_error')
lgsssupscores=cross_val_score(lgsssup.best_estimator_,sup_prepared,y_sup,cv=cv,scoring='neg_mean_squared_error')

    




/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

In [0]:
rfgsindneuscores=cross_val_score(rfindneu.best_estimator_,indneu_prepared,indy_neu,cv=cv,scoring='neg_mean_squared_error')
rfgsindproscores=cross_val_score(rfindpro.best_estimator_,indpro_prepared,indy_pro,cv=cv,scoring='neg_mean_squared_error')
rfgsindsupscores=cross_val_score(rfindsup.best_estimator_,indsup_prepared,indy_sup,cv=cv,scoring='neg_mean_squared_error')
svgsindneuscores=cross_val_score(svgsindneu.best_estimator_,indneu_prepared,indy_neu,cv=cv,scoring='neg_mean_squared_error')
svgsindproscores=cross_val_score(svgsindppro.best_estimator_,indpro_prepared,indy_pro,cv=cv,scoring='neg_mean_squared_error')
svgsindsupscores=cross_val_score(svgsindssup.best_estimator_,indsup_prepared,indy_sup,cv=cv,scoring='neg_mean_squared_error')
lgsindneuscores=cross_val_score(lgsindneu.best_estimator_,indneu_prepared,indy_neu,cv=cv,scoring='neg_mean_squared_error')
lgsindpproscores=cross_val_score(lgsindppro.best_estimator_,indpro_prepared,indy_pro,cv=cv,scoring='neg_mean_squared_error')
lgsindssupscores=cross_val_score(lgsindssup.best_estimator_,indsup_prepared,indy_sup,cv=cv,scoring='neg_mean_squared_error')



/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

In [0]:
rfgsmidneuscores=cross_val_score(rfmidneu.best_estimator_,midneu_prepared,midy_neu,cv=cv,scoring='neg_mean_squared_error')
rfgsmidproscores=cross_val_score(rfmidpro.best_estimator_,midpro_prepared,midy_pro,cv=cv,scoring='neg_mean_squared_error')
rfgsmidsupscores=cross_val_score(rfmidsup.best_estimator_,midsup_prepared,midy_sup,cv=cv,scoring='neg_mean_squared_error')
svgsmidneuscores=cross_val_score(svgsmidneu.best_estimator_,midneu_prepared,midy_neu,cv=cv,scoring='neg_mean_squared_error')
svgsmidproscores=cross_val_score(svgsmidppro.best_estimator_,midpro_prepared,midy_pro,cv=cv,scoring='neg_mean_squared_error')
svgsmidsupscores=cross_val_score(svgsmidssup.best_estimator_,midsup_prepared,midy_sup,cv=cv,scoring='neg_mean_squared_error')
lgsmidneuscores=cross_val_score(lgsmidneu.best_estimator_,midneu_prepared,midy_neu,cv=cv,scoring='neg_mean_squared_error')
lgsmidpproscores=cross_val_score(lgsmidppro.best_estimator_,midpro_prepared,midy_pro,cv=cv,scoring='neg_mean_squared_error')
lgsmidssupscores=cross_val_score(lgsmidssup.best_estimator_,midsup_prepared,midy_sup,cv=cv,scoring='neg_mean_squared_error')


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

In [0]:
vars=[rfgsneuscores,rfgspproscores,rfgssupscores,svgsneuscores,svgsproscores,svgssupscores,lgsneuscores,lgspproscores,lgsssupscores]

    
for var in vars:
  print(np.sqrt(-np.mean((var))))

7.7690854361368435
7.616554500904585
8.190458736849076
7.39808572266495
7.025126401377416
7.340093414857508
7.850047432893593
7.962010700937177
9.777674256666097


In [0]:
vars=[rfgsindneuscores,rfgsindproscores,rfgsindsupscores,svgsindneuscores,svgsindproscores,svgsindsupscores,lgsindneuscores,lgsindpproscores,lgsindssupscores]
    
for var in vars:
  print(np.sqrt(-np.mean((var))))

7.503609790985596
7.1921168362424055
5.374293940935321
7.460253499136508
6.89601851894246
5.136534782103607
7.513441207992463
8.433182719778118
6.020710905514466


In [0]:
vars=[rfgsmidneuscores,rfgsmidproscores,rfgsmidsupscores,svgsmidneuscores,svgsmidproscores,svgsmidsupscores,lgsmidneuscores,lgsmidpproscores,lgsmidssupscores]

for var in vars:
  print(np.sqrt(-np.mean((var))))

8.678418688870183
8.09955955630559
10.42984156010127
7.948396883406836
7.157552542649588
9.443923698915498
8.098594930480896
8.116230075258608
12.823565160409311


In [0]:
#best estimators
rfbestneu=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
svmbestneu=SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.0001, verbose=False)

lgsbestneu=LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,normalize=False)

rfbestind=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


svmbestind=SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.0001, verbose=False)

lgsbestind=LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)

rfbestmid=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


svmbestmid=SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)


lgsbestmid=LinearRegression(copy_X=True, fit_intercept='False', n_jobs=None,
                 normalize=False)



In [0]:
cv=KFold(n_splits=7)
rfgsneu=cross_val_score(rfbestneu,neu_prepared,y_neu,cv=cv)

In [0]:
#measure predictions

from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(a,b):
  return sqrt(mean_squared_error(a,b))

rfnp=rfgsneu.best_estimator_.predict(pro_prepared)
rfns=rfgsneu.best_estimator_.predict(sup_prepared)
svnp=svgsneu.best_estimator_.predict(pro_prepared)
svns=svgsneu.best_estimator_.predict(sup_prepared)
lmnp=lgsneu.best_estimator_.predict(pro_prepared)
lmns=lgsneu.best_estimator_.predict(sup_prepared)

rfinp=rfindneu.best_estimator_.predict(indpro_prepared)
rfins=rfindneu.best_estimator_.predict(indsup_prepared)
svinp=svgsindneu.best_estimator_.predict(indpro_prepared)
svins=svgsindneu.best_estimator_.predict(indsup_prepared)
lvinp=lgsindneu.best_estimator_.predict(indpro_prepared)
lvins=lgsindneu.best_estimator_.predict(indsup_prepared)

rfmnp=rfmidneu.best_estimator_.predict(midpro_prepared)
rfmns=rfmidneu.best_estimator_.predict(midsup_prepared)
svmnp=svgsmidneu.best_estimator_.predict(midpro_prepared)
svmns=svgsmidneu.best_estimator_.predict(midsup_prepared)
lvmnp=lgsmidneu.best_estimator_.predict(midpro_prepared)
lvmns=lgsmidneu.best_estimator_.predict(midsup_prepared)

In [0]:
print(rmse(rfnp,y_pro))
print(rmse(rfns,y_sup))
print(rmse(rfinp,indy_pro))
print(rmse(rfins,indy_sup))
print(rmse(rfmnp,midy_pro))
print(rmse(rfmns,midy_pro))

print(rmse(svnp,y_pro))
print(rmse(svns,y_sup))
print(rmse(svinp,indy_pro))
print(rmse(svins,indy_sup))
print(rmse(svmnp,midy_pro))
print(rmse(svmns,midy_sup))

print(rmse(lmnp,y_pro))
print(rmse(lmns,y_sup))
print(rmse(lvinp,indy_pro))
print(rmse(lvins,indy_sup))
print(rmse(lvmnp,midy_pro))
print(rmse(lvmns,midy_pro))

6.781884717221241
7.382109089856155
6.800616531026951
6.105253994445302
6.672458814253702
8.42652238427476
6.302186841175117
7.018458065143902
6.434345067405583
5.051135250755344
6.598199702624889
8.479364984824034
7.38499488294028
8.221304382638726
7.295687714741826
5.694399166529107
7.2736660446468475
10.733253853604165


In [0]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge


In [0]:

krr = KernelRidge()
KRR_params = {'alpha':[1,2],'kernel':['rbf'],'gamma':[.01,.1]}
krrneu=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrneu.fit(neu_prepared,y_neu)
krrppro=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrppro.fit(pro_prepared,y_pro)
krrssup=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrssup.fit(sup_prepared,y_sup)



GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None,
                                   kernel='linear', kernel_params=None),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.01, 0.1, 1], 'gamma': [0.01, 0.1],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(krrneu.best_estimator_)
print(krrppro.best_estimator_)
print(krrssup.best_estimator_)
print(krrneu.best_score_)
print(krrppro.best_score_)
print(krrssup.best_score_)

KernelRidge(alpha=1, coef0=1, degree=3, gamma=0.01, kernel='rbf',
            kernel_params=None)
KernelRidge(alpha=1, coef0=1, degree=3, gamma=0.01, kernel='rbf',
            kernel_params=None)
KernelRidge(alpha=1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
0.6679438775040328
0.6656749581913056
0.6608960532854535


In [0]:
lasso = Lasso()
cv=KFold(n_splits=7)
lasso_parameters = {'alpha':[1,2],'tol':[.00001,.000001],'max_iter':[1000,2000],'random_state':[41]}
lssneu=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssneu.fit(neu_prepared,y_neu)
lssppro=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssppro.fit(pro_prepared,y_pro)
lssssup=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssssup.fit(sup_prepared,y_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1, 2], 'max_iter': [1000, 2000],
                         'random_state': [41], 'tol': [1e-05, 1e-06]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(lssneu.best_estimator_)
print(lssppro.best_estimator_)
print(lssssup.best_estimator_)
print(lssneu.best_score_)
print(lssppro.best_score_)
print(lssssup.best_score_)

Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=41, selection='cyclic',
      tol=1e-06, warm_start=False)
Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=41, selection='cyclic',
      tol=1e-06, warm_start=False)
Lasso(alpha=2, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=41, selection='cyclic',
      tol=1e-06, warm_start=False)
0.6129638740577374
0.5905543140309036
0.48033822821464794


In [0]:
##
ENet = ElasticNet()
ENet_parameters = {'alpha':[0.1,.5,1],'l1_ratio':[.3,.5,0.8],'max_iter':[1000,2000],'random_state':[41]}
enetneu=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetneu.fit(neu_prepared,y_neu)
enetppro=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetppro.fit(pro_prepared,y_pro)
enetssup=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetssup.fit(sup_prepared,y_sup)
##

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1, 0.5, 1], 'l1_ratio': [0.3, 0.5, 0.8],
                         'max_iter': [1000, 2000], 'random_state': [41]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(enetneu.best_estimator_)
print(enetppro.best_estimator_)
print(enetssup.best_estimator_)
print(enetneu.best_score_)
print(enetppro.best_score_)
print(enetssup.best_score_)

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.8,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=41, selection='cyclic', tol=0.0001, warm_start=False)
ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.8,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=41, selection='cyclic', tol=0.0001, warm_start=False)
ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=0.8,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=41, selection='cyclic', tol=0.0001, warm_start=False)
0.611262038683224
0.5852141044447088
0.47961272589952


In [0]:
#next models index finger
krr = KernelRidge()
KRR_params = {'alpha':[1,2],'kernel':('rbf', 'polynomial')}
krrindneu=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrindneu.fit(indneu_prepared,indy_neu)
krrindppro=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrindppro.fit(indpro_prepared,indy_pro)
krrindssup=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrindssup.fit(indsup_prepared,indy_sup)


GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None,
                                   kernel='linear', kernel_params=None),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1, 2], 'kernel': ('rbf', 'polynomial')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(krrindneu.best_estimator_)
print(krrindppro.best_estimator_)
print(krrindssup.best_estimator_)
print(krrindneu.best_score_)
print(krrindppro.best_score_)
print(krrindssup.best_score_)

KernelRidge(alpha=2, coef0=1, degree=3, gamma=None, kernel='rbf',
            kernel_params=None)
KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='rbf',
            kernel_params=None)
KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='rbf',
            kernel_params=None)
0.535592227165765
0.6578395257017273
0.8252069467859573


In [0]:
lasso = Lasso()
lasso_parameters = {'alpha':[0.1,1],'tol':[.001,.0001,.00001],'random_state':[41]}
lssindneu=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssindneu.fit(indneu_prepared,indy_neu)
lssindppro=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssindppro.fit(indpro_prepared,indy_pro)
lssindssup=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssindssup.fit(indsup_prepared,indy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1, 1], 'random_state': [41],
                         'tol': [0.001, 0.0001, 1e-05]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(lssindneu.best_estimator_)
print(lssindppro.best_estimator_)
print(lssindssup.best_estimator_)
print(lssindneu.best_score_)
print(lssindppro.best_score_)
print(lssindssup.best_score_)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=41,
      selection='cyclic', tol=0.0001, warm_start=False)
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=41,
      selection='cyclic', tol=0.0001, warm_start=False)
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=41,
      selection='cyclic', tol=1e-05, warm_start=False)
0.6480523962619321
0.5446458610075995
0.7608964127669134


In [0]:
ENet = ElasticNet()
ENet_parameters = {'alpha':[0.001,0.01,0.1,1],'l1_ratio':[0.8,1],'random_state':[41]}
enetindneu=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetindneu.fit(indneu_prepared,indy_neu)
enetindppro=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetindppro.fit(indpro_prepared,indy_pro)
enetindssup=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetindssup.fit(indsup_prepared,indy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1], 'l1_ratio': [0.8, 1],
                         'random_state': [41]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(enetindneu.best_estimator_)
print(enetindppro.best_estimator_)
print(enetindssup.best_estimator_)
print(enetindneu.best_score_)
print(enetindppro.best_score_)
print(enetindssup.best_score_)

ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=1, max_iter=1000,
           normalize=False, positive=False, precompute=False, random_state=41,
           selection='cyclic', tol=0.0001, warm_start=False)
ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=1, max_iter=1000,
           normalize=False, positive=False, precompute=False, random_state=41,
           selection='cyclic', tol=0.0001, warm_start=False)
ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True, l1_ratio=0.8,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=41, selection='cyclic', tol=0.0001, warm_start=False)
0.6554999788220763
0.5699972876555857
0.7615013982786629


In [0]:
# mid finger
krr = KernelRidge()
KRR_params = {'alpha':[0.1,1,5],'kernel':('rbf', 'polynomial')}
krrmidneu=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrmidneu.fit(midneu_prepared,midy_neu)
krrmidppro=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrmidppro.fit(midpro_prepared,midy_pro)
krrmidssup=GridSearchCV(krr, KRR_params,cv = cv,scoring='r2')
krrmidssup.fit(midsup_prepared,midy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None,
                                   kernel='linear', kernel_params=None),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1, 1, 5], 'kernel': ('rbf', 'polynomial')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(krrmidneu.best_estimator_)
print(krrmidppro.best_estimator_)
print(krrmidssup.best_estimator_)
print(krrmidneu.best_score_)
print(krrmidppro.best_score_)
print(krrmidssup.best_score_)

In [0]:
lasso = Lasso()
lasso_parameters = {'alpha':[0.001,0.01,0.1],'tol':[.001,.0001,.00001],'random_state':[41]}
lssmidneu=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssmidneu.fit(midneu_prepared,midy_neu)
lssmidppro=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssmidppro.fit(midpro_prepared,midy_pro)
lssmidssup=GridSearchCV(lasso, lasso_parameters,cv = cv,scoring='r2')
lssmidssup.fit(midsup_prepared,midy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.001, 0.01, 0.1], 'random_state': [41],
                         'tol': [0.001, 0.0001, 1e-05]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(lssmidneu.best_estimator_)
print(lssmidppro.best_estimator_)
print(lssmidssup.best_estimator_)
print(lssmidneu.best_score_)
print(lssmidppro.best_score_)
print(lssmidssup.best_score_)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=41,
      selection='cyclic', tol=1e-05, warm_start=False)
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=41,
      selection='cyclic', tol=1e-05, warm_start=False)
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=41,
      selection='cyclic', tol=0.001, warm_start=False)
0.5893458732419163
0.5488863261590182
-0.10999807561252997


In [0]:
ENet = ElasticNet()
ENet_parameters = {'alpha':[0.001,0.01,0.1],'l1_ratio':[0.8,1],'random_state':[41]}
enetmidneu=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetmidneu.fit(midneu_prepared,midy_neu)
enetmidppro=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetmidppro.fit(midpro_prepared,midy_pro)
enetmidssup=GridSearchCV(ENet, ENet_parameters,cv = cv,scoring='r2')
enetmidssup.fit(midsup_prepared,midy_sup)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.001, 0.01, 0.1], 'l1_ratio': [0.8, 1],
                         'random_state': [41]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [0]:
print(enetmidneu.best_estimator_)
print(enetmidppro.best_estimator_)
print(enetmidssup.best_estimator_)
print(enetmidneu.best_score_)
print(enetmidppro.best_score_)
print(enetmidssup.best_score_)

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.8,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=41, selection='cyclic', tol=0.0001, warm_start=False)
ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.8,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=41, selection='cyclic', tol=0.0001, warm_start=False)
ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.8,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=41, selection='cyclic', tol=0.0001, warm_start=False)
0.5903499878654856
0.5581505028641012
-0.02649013730615341


In [0]:

krrneuscores=cross_val_score(krrneu.best_estimator_,neu_prepared,y_neu,cv=cv,scoring='neg_mean_squared_error')
krrpproscores=cross_val_score(krrppro.best_estimator_,pro_prepared,y_pro,cv=cv,scoring='neg_mean_squared_error')
krrssupscores=cross_val_score(krrssup.best_estimator_,sup_prepared,y_sup,cv=cv,scoring='neg_mean_squared_error')
lssneuscores=cross_val_score(lssneu.best_estimator_,neu_prepared,y_neu,cv=cv,scoring='neg_mean_squared_error')
lssproscores=cross_val_score(lssppro.best_estimator_,pro_prepared,y_pro,cv=cv,scoring='neg_mean_squared_error')
lssupscores=cross_val_score(lssssup.best_estimator_,sup_prepared,y_sup,cv=cv,scoring='neg_mean_squared_error')
enetneuscores=cross_val_score(enetneu.best_estimator_,neu_prepared,y_neu,cv=cv,scoring='neg_mean_squared_error')
enetproscores=cross_val_score(enetppro.best_estimator_,pro_prepared,y_pro,cv=cv,scoring='neg_mean_squared_error')
enetsupscores=cross_val_score(enetssup.best_estimator_,sup_prepared,y_sup,cv=cv,scoring='neg_mean_squared_error')


In [0]:
krrindneuscores=cross_val_score(krrindneu.best_estimator_,indneu_prepared,indy_neu,cv=cv,scoring='neg_mean_squared_error')
krrindpproscores=cross_val_score(krrindppro.best_estimator_,indpro_prepared,indy_pro,cv=cv,scoring='neg_mean_squared_error')
krrindssupscores=cross_val_score(krrindssup.best_estimator_,indsup_prepared,indy_sup,cv=cv,scoring='neg_mean_squared_error')
lssindneuscores=cross_val_score(lssindneu.best_estimator_,indneu_prepared,indy_neu,cv=cv,scoring='neg_mean_squared_error')
lssindproscores=cross_val_score(lssindppro.best_estimator_,indpro_prepared,indy_pro,cv=cv,scoring='neg_mean_squared_error')
lssindsupscores=cross_val_score(lssindssup.best_estimator_,indsup_prepared,indy_sup,cv=cv,scoring='neg_mean_squared_error')
enetindneuscores=cross_val_score(enetindneu.best_estimator_,indneu_prepared,indy_neu,cv=cv,scoring='neg_mean_squared_error')
enetindproscores=cross_val_score(enetindppro.best_estimator_,indpro_prepared,indy_pro,cv=cv,scoring='neg_mean_squared_error')
enetindsupscores=cross_val_score(enetindssup.best_estimator_,indsup_prepared,indy_sup,cv=cv,scoring='neg_mean_squared_error')


In [0]:
krrmidneuscores=cross_val_score(krrmidneu.best_estimator_,midneu_prepared,midy_neu,cv=cv,scoring='neg_mean_squared_error')
krrmidpproscores=cross_val_score(krrmidppro.best_estimator_,midpro_prepared,midy_pro,cv=cv,scoring='neg_mean_squared_error')
krrmidssupscores=cross_val_score(krrmidssup.best_estimator_,midsup_prepared,midy_sup,cv=cv,scoring='neg_mean_squared_error')
lssmidneuscores=cross_val_score(lssmidneu.best_estimator_,midneu_prepared,midy_neu,cv=cv,scoring='neg_mean_squared_error')
lssmidproscores=cross_val_score(lssmidppro.best_estimator_,midpro_prepared,midy_pro,cv=cv,scoring='neg_mean_squared_error')
lssmidsupscores=cross_val_score(lssmidssup.best_estimator_,midsup_prepared,midy_sup,cv=cv,scoring='neg_mean_squared_error')
enetmidneuscores=cross_val_score(enetmidneu.best_estimator_,midneu_prepared,midy_neu,cv=cv,scoring='neg_mean_squared_error')
enetmidproscores=cross_val_score(enetmidppro.best_estimator_,midpro_prepared,midy_pro,cv=cv,scoring='neg_mean_squared_error')
enetmidsupscores=cross_val_score(enetmidssup.best_estimator_,midsup_prepared,midy_sup,cv=cv,scoring='neg_mean_squared_error')


In [0]:
vars=[krrneuscores, krrpproscores,krrssupscores,lssneuscores,lssproscores,lssupscores,enetneuscores,enetproscores,enetsupscores]

    
for var in vars:
  print(np.sqrt(-np.mean((var))))

In [0]:
vars=[krrindneuscores,krrindpproscores,krrindssupscores,lssindneuscores,lssindproscores,lssindsupscores,enetindneuscores,enetindproscores,enetindsupscores]
    
for var in vars:
  print(np.sqrt(-np.mean((var))))

In [0]:
vars=[krrmidneuscores,krrmidpproscores,krrmidssupscores,lssmidneuscores,lssmidproscores,lssmidsupscores,enetmidneuscores,enetmidproscores,enetmidsupscores]
    
for var in vars:
  print(np.sqrt(-np.mean((var))))

In [0]:
krrnp=krrneu.best_estimator_.predict(pro_prepared)
krrns=krrneu.best_estimator_.predict(sup_prepared)
lssnp=lssneu.best_estimator_.predict(pro_prepared)
lssns=lssneu.best_estimator_.predict(sup_prepared)
enetnp=enetneu.best_estimator_.predict(pro_prepared)
enetns=enetneu.best_estimator_.predict(sup_prepared)

krrinp=krrindneu.best_estimator_.predict(indpro_prepared)
krrins=krrindneu.best_estimator_.predict(indsup_prepared)
lssinp=lssindneu.best_estimator_.predict(indpro_prepared)
lssins=lssindneu.best_estimator_.predict(indsup_prepared)
enetinp=enetindneu.best_estimator_.predict(indpro_prepared)
enetins=enetindneu.best_estimator_.predict(indsup_prepared)

krrmnp=krrmidneu.best_estimator_.predict(midpro_prepared)
krrmns=krrmidneu.best_estimator_.predict(midsup_prepared)
lssmnp=lssmidneu.best_estimator_.predict(midpro_prepared)
lssmns=lssmidneu.best_estimator_.predict(midsup_prepared)
enetmnp=enetmidneu.best_estimator_.predict(midpro_prepared)
enetmns=enetmidneu.best_estimator_.predict(midsup_prepared)

In [0]:
print(rmse(krrnp,y_pro))
print(rmse(krrns,y_sup))
print(rmse(krrinp,indy_pro))
print(rmse(krrins,indy_sup))
print(rmse(krrmnp,midy_pro))
print(rmse(krrmns,midy_pro))

print(rmse(lssnp,y_pro))
print(rmse(lssns,y_sup))
print(rmse(lssinp,indy_pro))
print(rmse(lssins,indy_sup))
print(rmse(lssmnp,midy_pro))
print(rmse(lssmns,midy_sup))

print(rmse(enetnp,y_pro))
print(rmse(enetns,y_sup))
print(rmse(enetinp,indy_pro))
print(rmse(enetins,indy_sup))
print(rmse(enetmnp,midy_pro))
print(rmse(enetmns,midy_pro))